In [2]:
from pyspark.sql import SparkSession,Row,DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *


# Create SparkSession
spark=SparkSession.builder.master("local[1]")\
        .appName("Music")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark/jars/mysql-connector-j-8.2.0.jar" ) \
        .getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

spark

24/01/15 21:33:54 WARN Utils: Your hostname, Eduardos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.3.5 instead (on interface en0)
24/01/15 21:33:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/15 21:33:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


PySpark Version :3.5.0
PySpark Version :3.5.0


In [40]:
def music(df: DataFrame ) -> DataFrame:
    df = df.selectExpr("cast(acousticness as decimal(3,2)) as acousticness",
                       "regexp_replace(artists,'[^a-zA-Z0-9]',' ') as artists",
                       "cast(danceability as decimal(3,2)) as danceability ",
                       "duration_ms",
                       "cast(energy as decimal(3,2)) as energy",
                       "explicit",
                       "cast(instrumentalness as decimal(3,2) ) as instrumentalness",
                       "key",
                       "cast (liveness as decimal(3,2) ) as liveness",
                       "mode",
                       "cast(loudness as decimal(3,2)) as loudness",
                       "name",
                       "popularity",
                       """ case
                              when release_date = '1928' then '1928-01-01' 
                              else release_date
                            end as release_date
                       """,
                       "cast(speechiness as decimal(5,2)) as speechiness",
                       "cast(tempo as decimal(3,2)) as tempo",
                       "cast(valence as decimal(3,2)) as valence",
                       "year")
    
    df = df.withColumn("rowid", monotonically_increasing_id())
    
    return df


def genre(df: DataFrame ) -> DataFrame:
    df = df.selectExpr("cast(danceability as decimal(3,2)) as danceability ",
                       "cast(energy as decimal(3,2)) as energy ",
                       "key ",
                       "cast(loudness as decimal(4,2)) as loudness ",
                       "mode",
                       "cast(speechiness as decimal(3,2)) as speechiness ",
                       "cast(acousticness as decimal(3,2)) as acousticness ",
                       "cast(instrumentalness as decimal(3,2)) as instrumentalness ",
                       "cast(liveness as decimal(3,2)) as liveness ",
                       "cast(valence as decimal(3,2)) as valence ",
                       "cast(tempo as decimal(3,2)) as tempo ",
                       "type",
                       "uri",
                       "track_href",
                       "analysis_url",
                       "duration_ms",
                       "time_signature",
                       "genre",
                       "song_name")
    df = df.withColumn("rowid", monotonically_increasing_id())

    return df

def  artists(df: DataFrame ) -> DataFrame:
    
    df = df.selectExpr("followers",
                       "regexp_replace(genres,'[^a-zA-Z0-9]',' ') as genres",
                       "name",
                       "popularity"

    )
    df = df.withColumn("id", row_number().over(Window.partitionBy("name").orderBy("name")))
    # df = df.selectExpr("id",
    #                    "genres",
    #                    "regexp_replace(name, r'\'','') as name",
    #                    "regexp_replace(popularity, r'\'','') as popularity",
    #                    "followers"
    # )
    # df = df.withColumn("rowid", monotonically_increasing_id())\
    #        .withColumn("teste", lit(1)+ (df["rowid"]-1))

    return df

    
    

    

In [43]:
txt = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/data.csv')    
genres = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/genres_v2.csv')
artist = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/artists.csv')

df = music(txt)
df.show(truncate=False)

df = genre(genres)
df.show(truncate=False)

df = artists(artist)
df.show(truncate=False)




+------------+-------------------------------------------------------------------+------------+-----------+------+--------+----------------+---+--------+----+--------+---------------------------------------------------------------------------+----------+------------+-----------+-----+-------+----+-----+
|acousticness|artists                                                            |danceability|duration_ms|energy|explicit|instrumentalness|key|liveness|mode|loudness|name                                                                       |popularity|release_date|speechiness|tempo|valence|year|rowid|
+------------+-------------------------------------------------------------------+------------+-----------+------+--------+----------------+---+--------+----+--------+---------------------------------------------------------------------------+----------+------------+-----------+-----+-------+----+-----+
|1.00        |  Carl Woitschach                                                  |0.7